In [1]:
from cmath import isnan
import os
from random import random
import time
from typing import Any
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer

# read the data from a file
data = pd.read_csv(os.getcwd().replace('code/old_test_code','') + "/data/cleaned_prof_data_v4.csv")

# initialize web driver for selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)


def findMaxCitations(web_elements: list):
    indOfMax = 0
    maxVal = 0
    for i in range(len(web_elements)):
        val = int(web_elements[i].text.split(" - ")[2].split(" ")[2])
        if val > maxVal:
            maxVal = val
            indOfMax = i
        else:
            continue
    return web_elements[indOfMax]


# loop through and find all professors with wrong / no google scholar ID
professors = []
professorExists = True

for i in range(13595, data.shape[0]):

    # initialize values
    scholar_id = data.iat[i, 3],
    citations = data.iat[i, 4],
    h_index = data.iat[i, 5]

    if(data.iat[i, 3] == "NOSCHOLARPAGE" or data.iat[i, 3] == "#NAME?" or isnan(data.iat[i, 4]) or isnan(data.iat[i, 5])):
        print("Name: " + data.iat[i, 0] + " | " + "University: " +
              data.iat[i, 1] + " | " + "id: " + data.iat[i, 3])

        # launch URL
        driver.get("https://scholar.google.com")

        # identify search box
        box = driver.find_element(By.NAME, "q")

        # enter search text
        box.send_keys(data.iat[i, 0] + " " + data.iat[i, 1])

        # make search
        box.send_keys(Keys.ENTER)

        # in the case that the author pops up as a recommended option
        try:
            link = driver.find_element(
                By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]/h4/a")
            link.click()
            time.sleep(random() * 3)
            
        except:
            try:
                # in the case there are multiple options in the recommended table
                list_of_elements = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]").find_elements(By.TAG_NAME, "div")
                print("case 2 was executed")
                max_element = findMaxCitations(list_of_elements)
                max_element.find_element(By.TAG_NAME, "a").click()
                time.sleep(random() * 5)
            except:
                # otherwise loop through all the divs and find the correct professor
                elements_list = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]").find_element(By.CLASS_NAME, "gs_a").find_elements(By.TAG_NAME, "a")
                print(elements_list)
                if elements_list: 
                    for element in elements_list:  # looping through the number of professors in the link
                        try:
                            element.find_element(By.TAG_NAME, "b")
                            element.click()
                            professorExists = True
                            break
                        except NoSuchElementException:
                            professorExists = False
                            continue  # in the case that the element is not found
                    # give time for html / css / js to load
                    time.sleep(random() * 5)
                else: 
                    professorExists = False 

        # retrieve citation, h-index, and scholar_id for that prof
        if professorExists:
            scholar_id = driver.current_url.split("=")[1].split("&")[0]
            citation = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[1]/td[2]").text
            h_index = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[2]/td[2]").text
            time.sleep(random() * 5)
        else:
            scholar_id = None
            citation = None
            h_index = None
        print("scholar_id: " + str(scholar_id) + " h-index: " +
              str(h_index) + " citations: " + str(citation))
        professors.append({
            "Name": data.iat[i, 0],
            "affiliation": data.iat[i, 1],
            "homepage": data.iat[i, 2],
            "scholar-id": scholar_id,
            "citations": citation,
            "h-index": h_index
        })
    else:
        continue


print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5_34.csv")

[WDM] - ====== WebDriver manager ======


2022-07-20 17:17:06,854 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-20 17:17:06,952 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="0e281b0a-1edc-4eb2-9b36-367e7b0bf5fa")>, <selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="d2b55eb9-c1a9-4555-bc74-a5c454a1302b")>]
scholar_id: g2bTiKAAAAAJ h-index: 13 citations: 635
Name: Michael Meier | University: University of Bonn | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="6e2b9df9-9bc9-488c-b2a4-0269aaf05305")>]
scholar_id: None h-index: None citations: None
Name: Norbert Blum | University: University of Bonn | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Joachim K. Anlauf | University: University of Bonn | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Matthew Smith | University: University of Bonn | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829

scholar_id: None h-index: None citations: None
Name: Hernán Wilkinson | University: University of Buenos Aires | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="c34f926f-e244-4274-9975-38d0357200bc")>, <selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="f76cc26b-bf0b-4787-beae-95c362df45e8")>, <selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="1d51ca41-f833-47d4-88e9-1c39fdea8881")>]
scholar_id: None h-index: None citations: None
Name: Pablo G. Turjanski | University: University of Buenos Aires | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="c67cfc89-0ca3-4f26-9c6a-dceeca1bb3a5")>]
scholar_id: None h-index: None citations: None
Name: Francisco J. Soulignac | University: University of Buenos Aires | id: NOSCHOLARPAGE
[<selenium.webdriver.

[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="e379e1d4-6749-41d2-95a9-7e3b4354e1eb")>, <selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="81a71169-bd82-42cc-bcca-4f1c08b88b19")>, <selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="8b281501-c496-44cb-aae2-5f27ef73f32d")>]
scholar_id: 0uLCV2EAAAAJ h-index: 31 citations: 2907
Name: Paula Buttery | University: University of Cambridge | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="e05a6f14-05b9-44bc-b25b-23c597ae6a68")>]
scholar_id: None h-index: None citations: None
Name: Sean B. Holden | University: University of Cambridge | id: NOSCHOLARPAGE
scholar_id: None h-index: None citations: None
Name: Damon Wischik | University: University of Cambridge | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.W

[]
scholar_id: None h-index: None citations: None
Name: Yuxin Chen | University: University of Chicago | id: #NAME?
scholar_id: None h-index: None citations: None
Name: Ketan D. Mulmuley | University: University of Chicago | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="5a62af37-72a4-45f8-b6f3-71bc9407dd13")>]
scholar_id: None h-index: None citations: None
Name: Yali Amit | University: University of Chicago | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="286120ca4829e945b447b4f27daef7f8", element="7c078bf5-dab6-4e0c-a844-7aa3fdde7fc7")>]
scholar_id: None h-index: None citations: None
Name: Robert I. Soare | University: University of Chicago | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Alexander A. Razborov | University: University of Chicago | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Ken Nakagaki | University: Univer

MaxRetryError: HTTPConnectionPool(host='localhost', port=60552): Max retries exceeded with url: /session/286120ca4829e945b447b4f27daef7f8/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11c0e6d60>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [2]:
df

NameError: name 'df' is not defined

In [3]:
professors

[{'Name': 'Michael Hatzopoulos',
  'affiliation': 'University of Athens',
  'homepage': 'http://cgi.di.uoa.gr/~mike',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Aggeliki Arapoyanni',
  'affiliation': 'University of Athens',
  'homepage': 'http://www.di.uoa.gr/eng/staff/1012',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Alex Delis',
  'affiliation': 'University of Athens',
  'homepage': 'http://cgi.di.uoa.gr/~ad',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Clark D. Thomborson',
  'affiliation': 'University of Auckland',
  'homepage': 'https://www.cs.auckland.ac.nz/~cthombor',
  'scholar-id': 'bSpUhd0AAAAJ',
  'citations': '12400',
  'h-index': '39'},
 {'Name': 'André Nies',
  'affiliation': 'University of Auckland',
  'homepage': 'https://www.cs.auckland.ac.nz/~nies',
  'scholar-id': 'zVPfH-QAAAAJ',
  'citations': '4670',
  'h-index': '32'},
 {'Name': 'David Welch',
  'affiliation': 'University o

In [15]:
h_index

54.0

In [3]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,P. Vassalos,AUEB,http://pubs.dbs.uni-leipzig.de/aa/aff/Greece/A...,NOSCHOLARPAGE,NaN,NaN
1,Stavros Toumpis,AUEB,https://195.251.255.130/en/faculty/toumpis-sta...,LyiVPKEAAAAJ,2804.0,21.0
2,Ion Androutsopoulos,AUEB,http://www.aueb.gr/users/ion,4UJm5EQAAAAJ,11918.0,42.0
3,Michalis K. Titsias,AUEB,http://www.aueb.gr/users/mtitsias,B-SbkAwAAAAJ,5255.0,30.0
4,Vasilios A. Siris,AUEB,http://www.aueb.gr/users/vsiris,Vj3O0cEAAAAJ,5967.0,31.0
...,...,...,...,...,...,...
20644,Ali C. Begen,Özyeğin University,https://ali.begen.net,ZIZWsXoAAAAJ,5964.0,34.0
20645,Erhan Öztop,Özyeğin University,https://robotics.ozyegin.edu.tr/members/erhan-...,A8YEQMMAAAAJ,3702.0,28.0
20646,Yasar Safkan,Özyeğin University,https://cs.ozyegin.edu.tr/en/faculty/yasarsafkan,NOSCHOLARPAGE,NaN,NaN
20647,Hasan Sözer,Özyeğin University,https://faculty.ozyegin.edu.tr/hsozer,L3FuUX0AAAAJ,847.0,16.0


In [5]:
data.loc[data['name'] == 'Peter Slater']

,name,affiliation,homepage,scholar_id,citations,h-index
13423,Peter Slater,University of Alabama - Huntsville,https://www.uah.edu/science/departments/math/f...,c2Zg-x8AAAAJ,NaN,NaN


In [4]:
print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5_34.csv")

175


In [ ]:
professors.append({'Name': 'Serafeim Karampogias',
  'affiliation': 'University of Athens',
  'homepage': 'None',
  'scholar-id': 'None',
  'citations': 'None',
  'h-index': 'None'})